# tt_curve_data_v1_vs_v2

Counting distinct parent compounds.

### Configuration

In [1]:
# ChEMBL connection...

engine = create_engine(open('database.txt').read().strip())

## Get counts for distinct parent compounds

_NB_ Symbol and Species here are redundant with Target ChEMBL ID but are kept for information.

In [2]:
count_by_target_v1 = pd.read_sql_query("""
select
    a.symbol
  , a.species
  , a.target_chemblid
  , count(distinct case when a.active = 1 then a.parent_cmpd_chemblid end) as n_active
  , count(distinct a.parent_cmpd_chemblid) as n_total
from
  tt_curve_data_V1 a  
where
    a.exclude = 0
and a.nat <= 50 
group by
    a.symbol
  , a.species
  , a.target_chemblid
""", engine) # .set_index(['symbol', 'species', 'target_chemblid'])

count_by_target_v1.shape

(304, 5)

In [3]:
count_by_target_v2 = pd.read_sql_query("""
select
    a.symbol
  , a.species
  , a.target_chemblid
  , count(distinct case when a.active = 1 and a.nat <= 50 then a.parent_cmpd_chemblid end) as n_active
  , count(distinct a.parent_cmpd_chemblid) as n_total
from
  tt_curve_data_V2 a  
where
    a.exclude = 0
and a.nat <= 50 
group by
    a.symbol
  , a.species
  , a.target_chemblid
""", engine) # .set_index(['symbol', 'species', 'target_chemblid'])

count_by_target_v2.shape

(312, 5)

In [4]:
df = count_by_target_v1.merge(count_by_target_v2, on=['symbol', 'species', 'target_chemblid'], how='outer', suffixes=['_v1', '_v2']).fillna(0)

df.shape

(312, 7)

In [5]:
df.query("n_active_v1 != n_active_v2")

,symbol,species,target_chemblid,n_active_v1,n_total_v1,n_active_v2,n_total_v2


In [8]:
df.query("n_total_v1 != n_total_v2")

,symbol,species,target_chemblid,n_active_v1,n_total_v1,n_active_v2,n_total_v2
0,AR,Rat,CHEMBL3072,304,392,304,1181
1,AR,Human,CHEMBL1871,1432,1482,1432,2335
2,F2,Rat,CHEMBL3078,10,11,10,13
3,F2,Human,CHEMBL204,3755,4357,3755,6041
6,ACE,Rat,CHEMBL2625,294,313,294,359
7,ACE,Human,CHEMBL1808,465,525,465,653
9,AHR,Human,CHEMBL3201,172,194,172,428
11,CA2,Human,CHEMBL205,3746,3894,3746,5000
12,KDR,Human,CHEMBL279,4505,4797,4505,6209
13,KIT,Human,CHEMBL1936,800,820,800,1464
